# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams

This notebook implements the methods described in [Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4040303), to be presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University. Here is a link to the presentation's [slide deck](https://docs.google.com/presentation/d/15fxG3zoZSdUfmxFuVQZd2_MrDe1Z1rcsXwBDIvFIy3U/edit?usp=sharing), including image credits. 

Some of the data presented here differs slightly from that found in the current SSRN article. Neither the p value or effect size have changed when measured to one significant figure, however. The difference comes from the fact that since the posting of the current SSRN version, it was decided that the numbers here would be more representative of the method's performance if instead of using one run of the algo over a single order of the input data the process was conducted over 100,000 pseudo-random shuffles. The older version of this repo can be found [here](https://github.com/colarusso/free-response-scoring/tree/24fccbc22817ed72687b3b35e0dcc461f724dc0b).

Paper Summary

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.000001, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 0.997).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

A PDF did not extist for every exam as some exams were hand-written. Also, for the _Property Instructor A_ exam, the IDs of two PDFs did not macth any of the IDs provided in the score sheet. To address such issues, you will see that we merge the two data tables above using the default behavior of Panda's merge method (i.e., we kept only the intersection of ID, those found in both files). Consequently, we did not score the exams for which no PDF existed or those for which we could not find a score on the intructor-provided list of scores.

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/data/property_instructor_A/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/data/property_instructor_B/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/data/crim_instructor_E/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), at least three of the answer sets may be shared upon request. One of these is linked to an exam requiring instructor assent to be shared. I am in the process of reviewing and cleaning these data for PII. You can see those data that have been cleaned and shared without need for instructor assent [here](https://github.com/colarusso/free-response-scoring/tree/main/data) and track my progress [here](https://twitter.com/Colarusso/status/1494781654237425666?s=20&t=g0Zt5f3wDnVys6oH3nVSGQ).

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts_redacted.csv"%exam["folder"], encoding="utf-8")
        # load this exam's grades
        actual = pd.read_csv("%s/actual_redacted.csv"%exam["folder"], encoding="utf-8")
        
        # convert human grades to z scores if score==3
        if score==2:
            actual = z_scores_from_grades(actual)

        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])        

        # score the questions listed above
        output_df = grade(texts[texts["ID"].isin(actual["ID"])][exam["columns"]],model,normv,score,goal)
        
        # below we want the average word counts for questions
        # so we need a df with only the questions actually used here
        # i.e., the inner merge. output_df doesn't have the texts columns
        # so we'll do a second merge here real quick to facilitate that count
        texts_df = actual.merge(texts, on="ID")

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts_df["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]]

            # ideally the granularity of the machine and human scoring should be the same
            # to avoid false precision. It may be worth imposing this in some manner 
            # simmilar to this comment. 
            # df["%s_y"%qn] = round(df["%s_x"%qn].max()*(df["%s_y"%qn]/100))
            # another option would be to conpare rounded z scores from both samples 

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades, averaged over `runs`. Append this to our list of 
            # swaps needed to move from machine-scored ordering to that of the graded exams. 
            # we're shuffling before sorting to address the fact that the original order of
            # rows can effect the swaps as the order of actual scores could be different
            # within the bins provided by the machine grade 
            tmp = []
            for x in range(runs):
                # shuffle df then sort answers by the machine scores
                # this is needed becaue the `actual` rows can effect the number 
                # of swaps, so we want to get a represnative sample over `runs`
                # note the use of `sample.` This is where we shuffle the base df
                df = df.sample(frac=1).sort_values(by='%s_y'%qn, ascending=True)
                
                arr_ = df[df.columns[1]].values.copy()
                n = len(arr_)
                M_o = countSwaps(arr_, n)
                tmp.append(M_o)

            M_o = np.array(tmp).mean()
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)

            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)

    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")

    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
        elif score==2:
            print("leaving z scores in place")
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

def z_scores_from_grades(df):
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    for x in questions:
        df[x] = df[x].apply(z_score_tradional, args=(df,x))
    return df

def z_score_tradional(row,df,x):
    # calculate the z-score for a given score
    return ((row-df[x].mean())/df[x].std())

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

The parameters below corresponds to those used when all of the exam data was first run through the above scoring method, with the exception of `runs`. To avoid biasing parameter selection, prior work on the method did NOT use the exam data used here. Note: the below cell has been re-run since this first run. Additionally, the current version makes use of the average number of swaps over 100,000 instances to provide a better representation of the method's performance.

In [11]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,83,75,84
1,9,30,22,87,90,86
2,6,27,9,69,72,83
3,1,1,4,82,72,79
4,13,26,25,91,85,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 989.49187
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1443, 1268, 1366, 1553, 1500, 1536, 1539, 1520, 1552, 1692]
Pseudo-random, avergae swaps needed: 1499.2635

Q1
Mean Words: 2048
Algo score, swaps needed: 1021.53089
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1571, 1465, 1592, 1454, 1451, 1656, 1704, 1527, 1640, 1678]
Pseudo-random, avergae swaps needed: 1566.17636

Q2
Mean Words: 947
Algo score, swaps needed: 830.0088
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1613, 1580, 1844, 1543, 1525, 1582, 1591, 1568, 1722, 1800]
Pseudo-random, avergae swaps needed: 1560.1296

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,88,88
1,18.0,20.0,85,80
2,16.5,18.0,59,89
3,18.0,22.5,82,90
4,24.0,22.5,84,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1289.48074
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1720, 1798, 2013, 1667, 1782, 1924, 1862, 1968, 1765, 1982]
Pseudo-random, avergae swaps needed: 1838.16739

Q2
Mean Words: 1546
Algo score, swaps needed: 1319.44584
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1590, 1748, 1852, 1944, 1830, 1819, 1734, 1795, 1595, 1746]
Pseudo-random, avergae swaps needed: 1757.34217

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 907.96902
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1107, 1426, 1236, 1337, 1401, 1363, 1499, 1309, 1352, 1298]
Pseudo-random, avergae swaps needed: 1324.92768

Q2
Mean Words: 570
Algo score, swaps needed: 813.94498
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1364, 1021, 1305, 1289, 1253, 1232, 1409, 1082, 1201, 1286]
Pseudo-random, avergae swaps needed: 1218.8786

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,82,89,85
1,44,29,30,85,84,89
2,38,36,20,81,84,81
3,47,16,21,79,81,81
4,38,25,31,89,81,89


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1175.99573
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1379, 1537, 1473, 1516, 1339, 1517, 1414, 1812, 1550, 1472]
Pseudo-random, avergae swaps needed: 1472.90819

Q2
Mean Words: 1050
Algo score, swaps needed: 995.51885
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1470, 1362, 1436, 1587, 1452, 1512, 1431, 1652, 1542, 1189]
Pseudo-random, avergae swaps needed: 1449.98639

Q3
Mean Words: 839
Algo score, swaps needed: 769.45228
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1164, 1435, 1196, 1344, 1430, 1322, 1494, 1392, 1590, 1682]
Pseudo-random, avergae swaps needed: 1458.82732

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,88
1,13.5,13.0,88,87
2,20.0,29.0,87,84
3,11.5,13.0,82,83
4,20.5,18.0,88,89


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1777.98812
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2150, 1974, 2039, 2124, 2171, 2141, 1981, 2166, 2509, 2027]
Pseudo-random, avergae swaps needed: 2045.47484

Q2
Mean Words: 2137
Algo score, swaps needed: 1635.97317
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2172, 1922, 1833, 2016, 1910, 2160, 1967, 1853, 2123, 1696]
Pseudo-random, avergae swaps needed: 1993.8371

Swaps (Machine):
[989.49187, 1021.53089, 830.0088, 1289.48074, 1319.44584, 120.99806, 907.96902, 813.94498, 1175.99573, 995.51885, 769.45228, 1777.98812, 1635.97317]
N: 13 	Mean:  1049.8306423076922 	Var:  160972.3340560062
Avergae swaps (Pseudo-random):
[1499.2635, 1566.17636, 1560.1296, 1838.16739, 1757.34217, 182.50397, 1324.92768, 1218.8786, 1472.90819, 1449.98639, 1458.82732, 2045.47484, 1993.8371]
N: 13 	Mean:  1489.8787007692308 	Var:  198389.97046703586 

1 ) 1499.2635 > 989.49187 => 40.9659893571621
2 ) 1566.1

The following cells present a variety of parameter selections, including a mix of vector normalization, grade translations, and "best answer" selections.

In [11]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,85,81,85
1,9,30,22,87,89,85
2,6,27,9,70,70,84
3,1,1,4,82,70,79
4,13,26,25,89,84,85


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996.49536
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1671, 1460, 1597, 1517, 1522, 1355, 1455, 1262, 1350, 1684]
Pseudo-random, avergae swaps needed: 1499.24089

Q1
Mean Words: 2048
Algo score, swaps needed: 1054.97656
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1558, 1614, 1556, 1536, 1629, 1664, 1544, 1775, 1706, 1439]
Pseudo-random, avergae swaps needed: 1566.37967

Q2
Mean Words: 947
Algo score, swaps needed: 959.99928
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1865, 1442, 1419, 1654, 1565, 1708, 1463, 1554, 1833, 1597]
Pseudo-random, avergae swaps needed: 1560.11219

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,89
1,18.0,20.0,84,81
2,16.5,18.0,59,89
3,18.0,22.5,84,92
4,24.0,22.5,84,88


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1247.51816
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2095, 1674, 1984, 1755, 1783, 1854, 1793, 1932, 1892, 2014]
Pseudo-random, avergae swaps needed: 1836.97758

Q2
Mean Words: 1546
Algo score, swaps needed: 1213.96302
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1704, 1676, 1554, 1898, 1837, 1711, 1769, 1731, 1914, 1727]
Pseudo-random, avergae swaps needed: 1757.60162

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 908.99593
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1441, 1326, 1292, 1207, 1382, 1385, 1399, 1495, 1438, 1436]
Pseudo-random, avergae swaps needed: 1324.2908

Q2
Mean Words: 570
Algo score, swaps needed: 907.53553
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1302, 1224, 1228, 1212, 1220, 1287, 1288, 1193, 1060, 1047]
Pseudo-random, avergae swaps needed: 1219.37611

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,88,85
1,44,29,30,84,83,89
2,38,36,20,81,83,81
3,47,16,21,81,87,81
4,38,25,31,89,79,89


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174.52751
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1371, 1380, 1411, 1651, 1433, 1461, 1203, 1597, 1414, 1621]
Pseudo-random, avergae swaps needed: 1471.92107

Q2
Mean Words: 1050
Algo score, swaps needed: 1041.00997
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1187, 1411, 1248, 1665, 1519, 1384, 1498, 1421, 1553, 1191]
Pseudo-random, avergae swaps needed: 1449.88628

Q3
Mean Words: 839
Algo score, swaps needed: 803.93484
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1348, 1386, 1321, 1366, 1416, 1351, 1473, 1610, 1299, 1736]
Pseudo-random, avergae swaps needed: 1458.3196

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,92,87
1,13.5,13.0,88,89
2,20.0,29.0,90,90
3,11.5,13.0,81,83
4,20.5,18.0,88,89


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1687.98389
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2065, 2040, 2160, 2194, 1926, 2000, 2031, 2167, 1885, 1931]
Pseudo-random, avergae swaps needed: 2045.10225

Q2
Mean Words: 2137
Algo score, swaps needed: 1579.0484
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1934, 1971, 2039, 2053, 1956, 1995, 1934, 1991, 1990, 1874]
Pseudo-random, avergae swaps needed: 1993.59363

Swaps (Machine):
[996.49536, 1054.97656, 959.99928, 1247.51816, 1213.96302, 113.99849, 908.99593, 907.53553, 1174.52751, 1041.00997, 803.93484, 1687.98389, 1579.0484]
N: 13 	Mean:  1053.0759184615383 	Var:  135083.89835593582
Avergae swaps (Pseudo-random):
[1499.24089, 1566.37967, 1560.11219, 1836.97758, 1757.60162, 182.48657, 1324.2908, 1219.37611, 1471.92107, 1449.88628, 1458.3196, 2045.10225, 1993.59363]
N: 13 	Mean:  1489.6375584615384 	Var:  198293.09108420022 

1 ) 1499.24089 > 996.49536 => 40.28835418806775
2 ) 

In [12]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,83,80,83
1,9,30,22,83,87,83
2,6,27,9,67,67,83
3,1,1,4,80,70,77
4,13,26,25,87,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1019.55157
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1583, 1379, 1310, 1427, 1634, 1365, 1524, 1703, 1639, 1753]
Pseudo-random, avergae swaps needed: 1500.18912

Q1
Mean Words: 2048
Algo score, swaps needed: 1070.35668
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1423, 1601, 1573, 1606, 1516, 1437, 1605, 1660, 1389, 1464]
Pseudo-random, avergae swaps needed: 1566.5108

Q2
Mean Words: 947
Algo score, swaps needed: 1035.25334
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1484, 1545, 1677, 1591, 1578, 1569, 1681, 1668, 1611, 1382]
Pseudo-random, avergae swaps needed: 1559.76023

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,87
1,18.0,20.0,83,80
2,16.5,18.0,59,87
3,18.0,22.5,83,90
4,24.0,22.5,83,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1275.06393
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1701, 2103, 1681, 1770, 2106, 1524, 1921, 1645, 1793, 1897]
Pseudo-random, avergae swaps needed: 1837.29274

Q2
Mean Words: 1546
Algo score, swaps needed: 1225.05197
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1620, 1688, 1697, 1806, 2014, 1828, 2118, 1717, 1787, 1958]
Pseudo-random, avergae swaps needed: 1756.62178

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 899.89604
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1416, 1259, 1251, 1405, 1149, 1315, 1390, 1228, 1367, 1151]
Pseudo-random, avergae swaps needed: 1324.19057

Q2
Mean Words: 570
Algo score, swaps needed: 1151.35415
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1207, 1140, 1301, 1394, 1120, 1267, 1235, 1280, 1337, 1266]
Pseudo-random, avergae swaps needed: 1219.02776

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,87,83
1,44,29,30,83,80,87
2,38,36,20,80,80,80
3,47,16,21,80,83,80
4,38,25,31,87,77,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1183.48457
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1582, 1460, 1385, 1459, 1408, 1675, 1458, 1177, 1374, 1478]
Pseudo-random, avergae swaps needed: 1472.49463

Q2
Mean Words: 1050
Algo score, swaps needed: 1061.53093
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1391, 1349, 1675, 1405, 1502, 1384, 1550, 1503, 1553, 1457]
Pseudo-random, avergae swaps needed: 1449.54714

Q3
Mean Words: 839
Algo score, swaps needed: 800.47716
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1551, 1545, 1362, 1421, 1529, 1489, 1494, 1359, 1452, 1474]
Pseudo-random, avergae swaps needed: 1458.0121

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,87,87
2,20.0,29.0,87,87
3,11.5,13.0,80,83
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1704.57395
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2193, 2094, 1919, 1842, 2027, 2243, 2009, 2026, 2090, 2257]
Pseudo-random, avergae swaps needed: 2045.56626

Q2
Mean Words: 2137
Algo score, swaps needed: 1642.38379
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1870, 2056, 2147, 1979, 2161, 2187, 1765, 2016, 1891, 1905]
Pseudo-random, avergae swaps needed: 1993.31859

Swaps (Machine):
[1019.55157, 1070.35668, 1035.25334, 1275.06393, 1225.05197, 115.50454, 899.89604, 1151.35415, 1183.48457, 1061.53093, 800.47716, 1704.57395, 1642.38379]
N: 13 	Mean:  1091.1140476923076 	Var:  140505.01265923667
Avergae swaps (Pseudo-random):
[1500.18912, 1566.5108, 1559.76023, 1837.29274, 1756.62178, 182.59659, 1324.19057, 1219.02776, 1472.49463, 1449.54714, 1458.0121, 2045.56626, 1993.31859]
N: 13 	Mean:  1489.6252546153846 	Var:  198284.13645084415 

1 ) 1500.18912 > 1019.55157 => 38.1497629424716

In [13]:
score_exams(exams,model='nlp1',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,0.120927,0.058564,0.057715,0.232847,-0.312513,0.231318
1,0.681318,0.676742,1.095029,0.477606,0.914631,0.310486
2,-0.159269,0.367653,-0.831411,-2.066348,-2.050004,0.066773
3,-1.560248,-2.311117,-1.572349,-0.147300,-1.927541,-0.591283
4,1.802101,0.264623,1.539592,0.835584,0.177698,0.284655


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 998.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1474, 1649, 1657, 1420, 1663, 1581, 1591, 1780, 1517, 1279]
Pseudo-random, avergae swaps needed: 1499.39333

Q1
Mean Words: 2048
Algo score, swaps needed: 1054.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1568, 1550, 1529, 1578, 1612, 1589, 1654, 1586, 1419, 1625]
Pseudo-random, avergae swaps needed: 1565.75904

Q2
Mean Words: 947
Algo score, swaps needed: 930.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1674, 1615, 1442, 1508, 1506, 1495, 1761, 1474, 1629, 1314]
Pseudo-random, avergae swaps needed: 1560.92914

../data/property_instructor_B

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,-0.393710,0.657180,0.528377,0.899438
1,0.314968,0.356847,0.104438,-0.388809
2,-0.039371,-0.243819,-4.844976,0.882998
3,0.314968,1.107680,0.164725,1.405384
4,1.732323,1.107680,0.074464,0.705205


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1275.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2002, 1901, 1904, 1739, 1531, 1835, 1492, 1870, 1723, 1922]
Pseudo-random, avergae swaps needed: 1837.61083

Q2
Mean Words: 1546
Algo score, swaps needed: 1224.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1640, 1703, 1748, 1807, 1917, 1646, 1447, 1825, 1807, 1855]
Pseudo-random, avergae swaps needed: 1758.00844

../data/environ_instructor_B

['ID', 'Q2']
leaving z scores in place


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 903.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1416, 1501, 1466, 1348, 1492, 1349, 1415, 1257, 1385, 1368]
Pseudo-random, avergae swaps needed: 1324.23303

Q2
Mean Words: 570
Algo score, swaps needed: 830.99768
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1293, 1320, 1115, 1097, 1019, 1244, 1267, 1114, 1351, 1191]
Pseudo-random, avergae swaps needed: 1220.04584

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,0.551646,1.374969,1.677993,-0.418742,0.696902,0.189555
1,-0.203612,0.246457,0.606719,0.150142,-0.076405,0.778135
2,-0.615570,0.964601,-0.367166,-0.361493,-0.021564,-0.413383
3,0.002368,-1.087240,-0.269777,-0.276170,0.458754,-0.270220
4,-0.615570,-0.163911,0.704108,0.860337,-0.746863,0.758854


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1176.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1527, 1400, 1351, 1286, 1651, 1295, 1387, 1611, 1475, 1387]
Pseudo-random, avergae swaps needed: 1471.85161

Q2
Mean Words: 1050
Algo score, swaps needed: 1043.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1388, 1349, 1548, 1549, 1508, 1278, 1439, 1645, 1338, 1605]
Pseudo-random, avergae swaps needed: 1450.08947

Q3
Mean Words: 839
Algo score, swaps needed: 783.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1425, 1670, 1348, 1309, 1448, 1248, 1428, 1561, 1381, 1535]
Pseudo-random, avergae swaps needed: 1458.26708

../data/crim_instructor_E

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,2.307804,-1.091625,1.347186,0.573983
1,-1.057500,-0.513174,0.623765,0.868420
2,0.400798,2.571898,0.997726,0.943209
3,-1.506207,-0.513174,-0.408669,0.055754
4,0.512975,0.450911,0.692992,0.816971


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1705.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2282, 1914, 2203, 1980, 2016, 2047, 2032, 1779, 1976, 1892]
Pseudo-random, avergae swaps needed: 2045.53001

Q2
Mean Words: 2137
Algo score, swaps needed: 1589.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2058, 1656, 2035, 2150, 1803, 2043, 2023, 2048, 2029, 1698]
Pseudo-random, avergae swaps needed: 1993.08202

Swaps (Machine):
[998.0, 1054.0, 930.0, 1275.0, 1224.0, 117.0, 903.0, 830.99768, 1176.0, 1043.0, 783.0, 1705.0, 1589.0]
N: 13 	Mean:  1048.3075138461538 	Var:  141907.9828879561
Avergae swaps (Pseudo-random):
[1499.39333, 1565.75904, 1560.92914, 1837.61083, 1758.00844, 182.53713, 1324.23303, 1220.04584, 1471.85161, 1450.08947, 1458.26708, 2045.53001, 1993.08202]
N: 13 	Mean:  1489.7951515384616 	Var:  198305.17492502136 

1 ) 1499.39333 > 998.0 => 40.153332995407666
2 ) 1565.75904 > 1054.0 => 39.06916874309173
3 ) 1560.92914 

In [14]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,80,80,83
1,9,30,22,87,87,80
2,6,27,9,67,67,83
3,1,1,4,80,70,80
4,13,26,25,83,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1242.43702
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1311, 1494, 1616, 1568, 1641, 1442, 1336, 1685, 1481, 1482]
Pseudo-random, avergae swaps needed: 1499.35637

Q1
Mean Words: 2048
Algo score, swaps needed: 1165.59888
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1593, 1628, 1654, 1484, 1407, 1701, 1682, 1559, 1604, 1607]
Pseudo-random, avergae swaps needed: 1566.40685

Q2
Mean Words: 947
Algo score, swaps needed: 1350.79614
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1513, 1527, 1556, 1515, 1353, 1630, 1406, 1676, 1707, 1663]
Pseudo-random, avergae swaps needed: 1559.81232

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,90
1,18.0,20.0,80,80
2,16.5,18.0,59,83
3,18.0,22.5,83,87
4,24.0,22.5,83,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1257.92074
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1600, 1897, 1699, 1922, 1852, 2064, 1796, 1804, 1967, 1627]
Pseudo-random, avergae swaps needed: 1837.85524

Q2
Mean Words: 1546
Algo score, swaps needed: 1579.04504
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1915, 1662, 1724, 2002, 1790, 1735, 1769, 1728, 1905, 1945]
Pseudo-random, avergae swaps needed: 1757.55048

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 914.45271
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1505, 1285, 1382, 1465, 1410, 1410, 1166, 1276, 1459, 1513]
Pseudo-random, avergae swaps needed: 1323.78049

Q2
Mean Words: 570
Algo score, swaps needed: 1132.22596
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1067, 1186, 1140, 1220, 1450, 1396, 1210, 1223, 1212, 1152]
Pseudo-random, avergae swaps needed: 1219.48789

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,83,83
1,44,29,30,80,77,87
2,38,36,20,77,83,80
3,47,16,21,83,87,80
4,38,25,31,87,73,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1299.59662
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1401, 1467, 1599, 1425, 1377, 1324, 1377, 1441, 1618, 1369]
Pseudo-random, avergae swaps needed: 1472.17812

Q2
Mean Words: 1050
Algo score, swaps needed: 1123.99405
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1615, 1584, 1363, 1608, 1297, 1376, 1444, 1413, 1415, 1580]
Pseudo-random, avergae swaps needed: 1449.90636

Q3
Mean Words: 839
Algo score, swaps needed: 924.12846
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1294, 1567, 1553, 1552, 1397, 1599, 1580, 1676, 1356, 1450]
Pseudo-random, avergae swaps needed: 1458.06519

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,87,83
1,13.5,13.0,83,87
2,20.0,29.0,87,87
3,11.5,13.0,80,83
4,20.5,18.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1882.37229
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1967, 2133, 1957, 2108, 1884, 2219, 2110, 2248, 1867, 1914]
Pseudo-random, avergae swaps needed: 2046.32073

Q2
Mean Words: 2137
Algo score, swaps needed: 1653.93677
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1965, 2007, 2026, 1927, 1831, 2037, 2201, 1928, 1875, 2204]
Pseudo-random, avergae swaps needed: 1993.52472

Swaps (Machine):
[1242.43702, 1165.59888, 1350.79614, 1257.92074, 1579.04504, 142.01671, 914.45271, 1132.22596, 1299.59662, 1123.99405, 924.12846, 1882.37229, 1653.93677]
N: 13 	Mean:  1205.270876153846 	Var:  164718.7575820746
Avergae swaps (Pseudo-random):
[1499.35637, 1566.40685, 1559.81232, 1837.85524, 1757.55048, 182.56695, 1323.78049, 1219.48789, 1472.17812, 1449.90636, 1458.06519, 2046.32073, 1993.52472]
N: 13 	Mean:  1489.7547469230772 	Var:  198426.67776564337 

1 ) 1499.35637 > 1242.43702 => 18.7409708504695

In [15]:
score_exams(exams,model='nlp1',normv=1,score=2,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,0.120927,0.058564,0.057715,-0.150690,-0.230911,0.030620
1,0.681318,0.676742,1.095029,0.826206,0.858392,-0.035948
2,-0.159269,0.367653,-0.831411,-2.131714,-2.186756,0.187587
3,-1.560248,-2.311117,-1.572349,-0.363009,-1.894981,-0.117108
4,1.802101,0.264623,1.539592,0.241292,0.225644,0.460836


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1204.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1380, 1443, 1512, 1457, 1438, 1487, 1239, 1492, 1550, 1567]
Pseudo-random, avergae swaps needed: 1499.20739

Q1
Mean Words: 2048
Algo score, swaps needed: 1169.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1487, 1473, 1458, 1637, 1573, 1487, 1591, 1444, 1550, 1703]
Pseudo-random, avergae swaps needed: 1565.99342

Q2
Mean Words: 947
Algo score, swaps needed: 1254.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1623, 1495, 1462, 1482, 1698, 1337, 1436, 1429, 1450, 1615]
Pseudo-random, avergae swaps needed: 1560.22353

../data/property_instructor_B

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,-0.393710,0.657180,0.845596,1.079657
1,0.314968,0.356847,-0.015124,-0.143833
2,-0.039371,-0.243819,-4.074838,0.410912
3,0.314968,1.107680,0.003355,0.501582
4,1.732323,1.107680,0.252583,0.108314


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1270.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1872, 1570, 1764, 1600, 1907, 1779, 1596, 2025, 1752, 1596]
Pseudo-random, avergae swaps needed: 1837.44116

Q2
Mean Words: 1546
Algo score, swaps needed: 1616.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1735, 1898, 1732, 1600, 1722, 1738, 1781, 1585, 1874, 1876]
Pseudo-random, avergae swaps needed: 1757.10609

../data/environ_instructor_B

['ID', 'Q2']
leaving z scores in place


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 917.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1470, 1332, 1507, 1131, 1257, 1255, 1254, 1131, 1189, 1542]
Pseudo-random, avergae swaps needed: 1324.3877

Q2
Mean Words: 570
Algo score, swaps needed: 875.98935
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1312, 1055, 1386, 1288, 1202, 1234, 1295, 1232, 1136, 1201]
Pseudo-random, avergae swaps needed: 1219.42682

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,0.551646,1.374969,1.677993,-0.023152,0.326200,0.233111
1,-0.203612,0.246457,0.606719,-0.074922,-0.658055,0.650828
2,-0.615570,0.964601,-0.367166,-0.820483,0.015432,-0.176209
3,0.002368,-1.087240,-0.269777,0.057478,0.738056,-0.273807
4,-0.615570,-0.163911,0.704108,0.905053,-1.119914,0.591653


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1273.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1595, 1678, 1484, 1604, 1457, 1479, 1446, 1616, 1501, 1489]
Pseudo-random, avergae swaps needed: 1472.22519

Q2
Mean Words: 1050
Algo score, swaps needed: 1090.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1678, 1476, 1511, 1383, 1395, 1556, 1355, 1378, 1422, 1434]
Pseudo-random, avergae swaps needed: 1449.69485

Q3
Mean Words: 839
Algo score, swaps needed: 874.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1513, 1315, 1585, 1503, 1484, 1513, 1570, 1238, 1334, 1341]
Pseudo-random, avergae swaps needed: 1458.40118

../data/crim_instructor_E

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,2.307804,-1.091625,0.749033,0.272315
1,-1.057500,-0.513174,0.047783,0.749472
2,0.400798,2.571898,0.783171,0.917546
3,-1.506207,-0.513174,-0.405370,0.178745
4,0.512975,0.450911,0.782507,0.131989


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1869.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1960, 2219, 2227, 2122, 1967, 1866, 1921, 1999, 2020, 2024]
Pseudo-random, avergae swaps needed: 2045.93679

Q2
Mean Words: 2137
Algo score, swaps needed: 1647.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1887, 2155, 1987, 1953, 1722, 2000, 2082, 2068, 1999, 2145]
Pseudo-random, avergae swaps needed: 1994.46441

Swaps (Machine):
[1204.0, 1169.0, 1254.0, 1270.0, 1616.0, 148.0, 917.0, 875.98935, 1273.0, 1090.0, 874.0, 1869.0, 1647.0]
N: 13 	Mean:  1169.7684115384616 	Var:  171755.27423704774
Avergae swaps (Pseudo-random):
[1499.20739, 1565.99342, 1560.22353, 1837.44116, 1757.10609, 182.58712, 1324.3877, 1219.42682, 1472.22519, 1449.69485, 1458.40118, 2045.93679, 1994.46441]
N: 13 	Mean:  1489.7765884615385 	Var:  198408.16942096938 

1 ) 1499.20739 > 1204.0 => 21.84126834604429
2 ) 1565.99342 > 1169.0 => 29.030667284018552
3 ) 1560.223

In [16]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,77,83,80
1,9,30,22,80,80,83
2,6,27,9,73,77,80
3,1,1,4,83,73,77
4,13,26,25,87,80,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1002.07985
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1459, 1528, 1484, 1304, 1617, 1492, 1417, 1233, 1212, 1579]
Pseudo-random, avergae swaps needed: 1498.97483

Q1
Mean Words: 2048
Algo score, swaps needed: 1192.92756
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1799, 1440, 1656, 1502, 1515, 1852, 1629, 1583, 1815, 1683]
Pseudo-random, avergae swaps needed: 1565.96733

Q2
Mean Words: 947
Algo score, swaps needed: 970.43511
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1838, 1528, 1660, 1596, 1613, 1546, 1687, 1603, 1591, 1695]
Pseudo-random, avergae swaps needed: 1560.11899

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,83
1,18.0,20.0,80,83
2,16.5,18.0,60,83
3,18.0,22.5,87,87
4,24.0,22.5,87,90


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1283.14119
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2213, 1990, 1951, 1751, 1671, 1885, 1814, 1846, 2039, 1944]
Pseudo-random, avergae swaps needed: 1837.6722

Q2
Mean Words: 1546
Algo score, swaps needed: 1229.95412
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1810, 1660, 1779, 1734, 1549, 1745, 1849, 1958, 1851, 1872]
Pseudo-random, avergae swaps needed: 1757.29505

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 919.00632
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1436, 1243, 1450, 1186, 1475, 1347, 1224, 1438, 1324, 1283]
Pseudo-random, avergae swaps needed: 1324.35694

Q2
Mean Words: 570
Algo score, swaps needed: 834.05294
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1195, 1182, 1068, 1274, 1245, 1290, 1046, 1114, 1262, 1283]
Pseudo-random, avergae swaps needed: 1219.14271

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,93,87
1,44,29,30,80,80,87
2,38,36,20,77,80,80
3,47,16,21,73,77,73
4,38,25,31,83,80,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1139.96847
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1546, 1286, 1376, 1342, 1680, 1541, 1423, 1322, 1429, 1406]
Pseudo-random, avergae swaps needed: 1472.62674

Q2
Mean Words: 1050
Algo score, swaps needed: 908.51111
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1353, 1602, 1398, 1631, 1337, 1381, 1458, 1494, 1492, 1475]
Pseudo-random, avergae swaps needed: 1449.24578

Q3
Mean Words: 839
Algo score, swaps needed: 857.67137
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1537, 1341, 1294, 1337, 1602, 1392, 1572, 1369, 1415, 1172]
Pseudo-random, avergae swaps needed: 1457.43887

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,80,87
2,20.0,29.0,90,83
3,11.5,13.0,77,83
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1746.12866
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2206, 2314, 2255, 2317, 1977, 1953, 1906, 1919, 1802, 1989]
Pseudo-random, avergae swaps needed: 2045.64958

Q2
Mean Words: 2137
Algo score, swaps needed: 1730.91976
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2023, 1895, 1924, 1976, 2317, 1960, 2045, 1908, 2036, 2044]
Pseudo-random, avergae swaps needed: 1993.99214

Swaps (Machine):
[1002.07985, 1192.92756, 970.43511, 1283.14119, 1229.95412, 90.99815, 919.00632, 834.05294, 1139.96847, 908.51111, 857.67137, 1746.12866, 1730.91976]
N: 13 	Mean:  1069.6765084615386 	Var:  162122.14957222383
Avergae swaps (Pseudo-random):
[1498.97483, 1565.96733, 1560.11899, 1837.6722, 1757.29505, 182.47992, 1324.35694, 1219.14271, 1472.62674, 1449.24578, 1457.43887, 2045.64958, 1993.99214]
N: 13 	Mean:  1489.6123907692306 	Var:  198405.8812341614 

1 ) 1498.97483 > 1002.07985 => 39.734835385526246
2

In [17]:
score_exams(exams,model='nlp2',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,0.120927,0.058564,0.057715,-0.643698,0.017657,-0.149936
1,0.681318,0.676742,1.095029,-0.300975,-0.285017,0.301280
2,-0.159269,0.367653,-0.831411,-1.346377,-0.915538,-0.361430
3,-1.560248,-2.311117,-1.572349,0.353406,-1.007537,-0.739357
4,1.802101,0.264623,1.539592,0.682897,-0.139262,0.671496


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1004.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1427, 1439, 1465, 1576, 1804, 1577, 1356, 1404, 1526, 1284]
Pseudo-random, avergae swaps needed: 1499.76064

Q1
Mean Words: 2048
Algo score, swaps needed: 1176.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1646, 1533, 1562, 1624, 1686, 1615, 1636, 1647, 1601, 1756]
Pseudo-random, avergae swaps needed: 1566.98541

Q2
Mean Words: 947
Algo score, swaps needed: 898.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1494, 1442, 1820, 1693, 1477, 1747, 1600, 1434, 1304, 1543]
Pseudo-random, avergae swaps needed: 1559.55872

../data/property_instructor_B

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,-0.393710,0.657180,0.565346,0.388619
1,0.314968,0.356847,-0.489128,0.231512
2,-0.039371,-0.243819,-3.360632,0.422747
3,0.314968,1.107680,0.928382,0.976120
4,1.732323,1.107680,0.783144,1.033481


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1271.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1821, 1896, 1848, 1687, 1889, 1857, 1957, 1952, 1699, 1888]
Pseudo-random, avergae swaps needed: 1838.20679

Q2
Mean Words: 1546
Algo score, swaps needed: 1196.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1709, 1738, 1639, 1600, 1702, 1828, 1498, 1775, 1971, 1787]
Pseudo-random, avergae swaps needed: 1757.67545

../data/environ_instructor_B

['ID', 'Q2']
leaving z scores in place


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 919.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1458, 1106, 1143, 1229, 1309, 1314, 1223, 1321, 1233, 1438]
Pseudo-random, avergae swaps needed: 1324.48541

Q2
Mean Words: 570
Algo score, swaps needed: 923.99184
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1205, 1232, 1278, 1143, 1150, 1160, 1217, 1062, 1278, 1298]
Pseudo-random, avergae swaps needed: 1219.69463

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,0.551646,1.374969,1.677993,0.405869,1.715751,0.950219
1,-0.203612,0.246457,0.606719,-0.080626,-0.415459,0.518973
2,-0.615570,0.964601,-0.367166,-0.519384,-0.485636,-0.248596
3,0.002368,-1.087240,-0.269777,-1.211684,-0.556331,-1.414347
4,-0.615570,-0.163911,0.704108,0.356245,-0.086010,0.871959


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1184.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1367, 1509, 1420, 1525, 1406, 1414, 1656, 1526, 1639, 1433]
Pseudo-random, avergae swaps needed: 1472.43231

Q2
Mean Words: 1050
Algo score, swaps needed: 905.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1529, 1606, 1363, 1546, 1534, 1193, 1412, 1317, 1413, 1537]
Pseudo-random, avergae swaps needed: 1449.77146

Q3
Mean Words: 839
Algo score, swaps needed: 831.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1490, 1493, 1362, 1623, 1497, 1298, 1363, 1317, 1558, 1371]
Pseudo-random, avergae swaps needed: 1458.23026

../data/crim_instructor_E

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,2.307804,-1.091625,1.347324,0.797131
1,-1.057500,-0.513174,-0.413074,0.730841
2,0.400798,2.571898,1.105897,0.134974
3,-1.506207,-0.513174,-0.549559,0.130096
4,0.512975,0.450911,0.681906,0.825864


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1729.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2229, 1864, 2050, 1995, 1942, 1888, 2070, 2371, 2059, 2176]
Pseudo-random, avergae swaps needed: 2045.76427

Q2
Mean Words: 2137
Algo score, swaps needed: 1753.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1944, 1919, 1968, 2059, 2117, 2114, 1988, 1786, 1869, 2046]
Pseudo-random, avergae swaps needed: 1993.70819

Swaps (Machine):
[1004.0, 1176.0, 898.0, 1271.0, 1196.0, 92.0, 919.0, 923.99184, 1184.0, 905.0, 831.0, 1729.0, 1753.0]
N: 13 	Mean:  1067.845526153846 	Var:  161546.7723029528
Avergae swaps (Pseudo-random):
[1499.76064, 1566.98541, 1559.55872, 1838.20679, 1757.67545, 182.52025, 1324.48541, 1219.69463, 1472.43231, 1449.77146, 1458.23026, 2045.76427, 1993.70819]
N: 13 	Mean:  1489.9072146153849 	Var:  198404.15010983663 

1 ) 1499.76064 > 1004.0 => 39.60128073584542
2 ) 1566.98541 > 1176.0 => 28.508019661686784
3 ) 1559.55872 >

In [18]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,77,80,83
1,9,30,22,77,77,80
2,6,27,9,73,73,80
3,1,1,4,80,77,77
4,13,26,25,83,80,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1244.62163
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1392, 1474, 1502, 1444, 1321, 1386, 1390, 1456, 1705, 1382]
Pseudo-random, avergae swaps needed: 1499.79449

Q1
Mean Words: 2048
Algo score, swaps needed: 1321.91847
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1525, 1367, 1357, 1354, 1510, 1554, 1590, 1601, 1669, 1499]
Pseudo-random, avergae swaps needed: 1566.74012

Q2
Mean Words: 947
Algo score, swaps needed: 1268.02826
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1495, 1519, 1438, 1311, 1424, 1531, 1662, 1664, 1643, 1550]
Pseudo-random, avergae swaps needed: 1559.94705

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,87
1,18.0,20.0,77,83
2,16.5,18.0,67,83
3,18.0,22.5,87,87
4,24.0,22.5,87,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1277.58662
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2144, 1923, 1874, 1791, 1695, 1876, 2079, 1982, 1985, 1759]
Pseudo-random, avergae swaps needed: 1837.86752

Q2
Mean Words: 1546
Algo score, swaps needed: 1150.38935
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1500, 1703, 1660, 1617, 1829, 1668, 1804, 1784, 1894, 1722]
Pseudo-random, avergae swaps needed: 1757.70438

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1007.51157
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1398, 1295, 1218, 1447, 1314, 1299, 1334, 1406, 1321, 1520]
Pseudo-random, avergae swaps needed: 1324.21144

Q2
Mean Words: 570
Algo score, swaps needed: 985.03686
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1295, 1223, 1249, 1280, 1270, 1169, 1078, 1368, 1100, 1135]
Pseudo-random, avergae swaps needed: 1219.43209

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,97,87
1,44,29,30,83,77,87
2,38,36,20,80,77,77
3,47,16,21,73,77,77
4,38,25,31,83,83,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1124.54767
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1500, 1473, 1607, 1426, 1272, 1324, 1404, 1655, 1400, 1567]
Pseudo-random, avergae swaps needed: 1472.41661

Q2
Mean Words: 1050
Algo score, swaps needed: 969.07747
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1267, 1624, 1492, 1310, 1646, 1353, 1581, 1390, 1591, 1445]
Pseudo-random, avergae swaps needed: 1449.55052

Q3
Mean Words: 839
Algo score, swaps needed: 775.02654
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1547, 1477, 1598, 1385, 1389, 1495, 1277, 1502, 1469, 1425]
Pseudo-random, avergae swaps needed: 1458.68701

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,87,83
1,13.5,13.0,83,83
2,20.0,29.0,87,83
3,11.5,13.0,73,83
4,20.5,18.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1790.59237
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2073, 2226, 2085, 2035, 1900, 1991, 2192, 2048, 1744, 2074]
Pseudo-random, avergae swaps needed: 2046.05244

Q2
Mean Words: 2137
Algo score, swaps needed: 1715.63224
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1579, 2156, 1939, 1839, 2160, 1943, 1988, 2068, 1987, 2005]
Pseudo-random, avergae swaps needed: 1994.16553

Swaps (Machine):
[1244.62163, 1321.91847, 1268.02826, 1277.58662, 1150.38935, 112.4981, 1007.51157, 985.03686, 1124.54767, 969.07747, 775.02654, 1790.59237, 1715.63224]
N: 13 	Mean:  1134.0359346153846 	Var:  161047.35303102978
Avergae swaps (Pseudo-random):
[1499.79449, 1566.74012, 1559.94705, 1837.86752, 1757.70438, 182.49595, 1324.21144, 1219.43209, 1472.41661, 1449.55052, 1458.68701, 2046.05244, 1994.16553]
N: 13 	Mean:  1489.9280884615384 	Var:  198470.58965656857 

1 ) 1499.79449 > 1244.62163 => 18.5957849569838

In [19]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,87,67,80
1,9,30,22,80,83,83
2,6,27,9,73,73,83
3,1,1,4,87,80,83
4,13,26,25,87,77,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1327.58895
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1416, 1617, 1441, 1269, 1431, 1332, 1539, 1372, 1615, 1531]
Pseudo-random, avergae swaps needed: 1500.02149

Q1
Mean Words: 2048
Algo score, swaps needed: 1284.03048
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1536, 1566, 1547, 1547, 1789, 1565, 1617, 1413, 1434, 1517]
Pseudo-random, avergae swaps needed: 1567.23205

Q2
Mean Words: 947
Algo score, swaps needed: 1217.32733
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1496, 1574, 1670, 1627, 1457, 1557, 1561, 1685, 1545, 1670]
Pseudo-random, avergae swaps needed: 1560.1602

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,90
1,18.0,20.0,60,67
2,16.5,18.0,63,83
3,18.0,22.5,77,87
4,24.0,22.5,87,90


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1578.60998
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1678, 1983, 1854, 2030, 1627, 1760, 2086, 1726, 1899, 1747]
Pseudo-random, avergae swaps needed: 1837.802

Q2
Mean Words: 1546
Algo score, swaps needed: 1572.44987
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1607, 1950, 2019, 1726, 1990, 1832, 1670, 1792, 1883, 1566]
Pseudo-random, avergae swaps needed: 1757.14286

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 926.48688
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1261, 1240, 1440, 1421, 1208, 1288, 1209, 1079, 1388, 1201]
Pseudo-random, avergae swaps needed: 1324.49526

Q2
Mean Words: 570
Algo score, swaps needed: 1010.99637
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1199, 1368, 1270, 1211, 1178, 1213, 1140, 1034, 1405, 1166]
Pseudo-random, avergae swaps needed: 1219.38382

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,87,90
1,44,29,30,77,73,83
2,38,36,20,87,83,83
3,47,16,21,60,67,59
4,38,25,31,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1355.53119
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1331, 1520, 1518, 1506, 1350, 1520, 1302, 1562, 1277, 1443]
Pseudo-random, avergae swaps needed: 1473.19422

Q2
Mean Words: 1050
Algo score, swaps needed: 1163.68601
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1364, 1424, 1522, 1229, 1415, 1475, 1401, 1416, 1686, 1420]
Pseudo-random, avergae swaps needed: 1449.83652

Q3
Mean Words: 839
Algo score, swaps needed: 1059.51561
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1390, 1593, 1424, 1616, 1501, 1386, 1417, 1503, 1431, 1506]
Pseudo-random, avergae swaps needed: 1457.7354

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,83,87
2,20.0,29.0,80,77
3,11.5,13.0,87,80
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1718.95664
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2018, 2107, 2221, 2146, 2248, 2127, 2061, 2106, 2085, 1918]
Pseudo-random, avergae swaps needed: 2045.70438

Q2
Mean Words: 2137
Algo score, swaps needed: 1802.01605
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1977, 1937, 2139, 1876, 1728, 2072, 1679, 2101, 1966, 1966]
Pseudo-random, avergae swaps needed: 1993.91772

Swaps (Machine):
[1327.58895, 1284.03048, 1217.32733, 1578.60998, 1572.44987, 108.50587, 926.48688, 1010.99637, 1355.53119, 1163.68601, 1059.51561, 1718.95664, 1802.01605]
N: 13 	Mean:  1240.438556153846 	Var:  174101.63047837283
Avergae swaps (Pseudo-random):
[1500.02149, 1567.23205, 1560.1602, 1837.802, 1757.14286, 182.49916, 1324.49526, 1219.38382, 1473.19422, 1449.83652, 1457.7354, 2045.70438, 1993.91772]
N: 13 	Mean:  1489.9326984615386 	Var:  198398.42354961592 

1 ) 1500.02149 > 1327.58895 => 12.196343425581636

In [20]:
score_exams(exams,model='nlp3',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,0.120927,0.058564,0.057715,0.556360,-2.092168,-0.147927
1,0.681318,0.676742,1.095029,-0.473255,0.295704,0.289554
2,-0.159269,0.367653,-0.831411,-1.323847,-1.126254,0.412401
3,-1.560248,-2.311117,-1.572349,0.687166,-0.057192,0.220752
4,1.802101,0.264623,1.539592,0.961585,-0.685142,0.678396


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1261.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1413, 1577, 1433, 1683, 1262, 1632, 1520, 1553, 1585, 1640]
Pseudo-random, avergae swaps needed: 1499.08496

Q1
Mean Words: 2048
Algo score, swaps needed: 1261.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1650, 1570, 1546, 1439, 1715, 1560, 1399, 1460, 1422, 1592]
Pseudo-random, avergae swaps needed: 1566.46133

Q2
Mean Words: 947
Algo score, swaps needed: 1165.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1858, 1575, 1765, 1504, 1412, 1438, 1414, 1595, 1689, 1550]
Pseudo-random, avergae swaps needed: 1560.01991

../data/property_instructor_B

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,-0.393710,0.657180,0.721736,1.016099
1,0.314968,0.356847,-3.137119,-2.034291
2,-0.039371,-0.243819,-2.621532,0.210490
3,0.314968,1.107680,-0.751679,0.583587
4,1.732323,1.107680,0.805917,1.421658


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1571.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1841, 1867, 1756, 1922, 1704, 1928, 1807, 2240, 1768, 2055]
Pseudo-random, avergae swaps needed: 1837.60356

Q2
Mean Words: 1546
Algo score, swaps needed: 1500.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1734, 1765, 1687, 1813, 1641, 1993, 1734, 1647, 1656, 1659]
Pseudo-random, avergae swaps needed: 1757.76941

../data/environ_instructor_B

['ID', 'Q2']
leaving z scores in place


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 891.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1367, 1175, 1333, 1186, 1479, 1272, 1447, 1194, 1389, 1344]
Pseudo-random, avergae swaps needed: 1324.54328

Q2
Mean Words: 570
Algo score, swaps needed: 1100.99688
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1196, 1090, 1255, 1180, 1214, 1208, 1185, 1260, 1351, 1298]
Pseudo-random, avergae swaps needed: 1219.78771

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']
leaving z scores in place
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,0.551646,1.374969,1.677993,0.074149,0.832997,1.097431
1,-0.203612,0.246457,0.606719,-0.695159,-1.376325,0.101474
2,-0.615570,0.964601,-0.367166,0.616868,0.135432,0.304646
3,0.002368,-1.087240,-0.269777,-3.389213,-2.288697,-3.737298
4,-0.615570,-0.163911,0.704108,0.251161,0.558974,0.948471


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1337.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1287, 1406, 1395, 1436, 1361, 1568, 1445, 1413, 1461, 1637]
Pseudo-random, avergae swaps needed: 1472.20256

Q2
Mean Words: 1050
Algo score, swaps needed: 1112.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1512, 1494, 1340, 1447, 1411, 1608, 1440, 1571, 1309, 1345]
Pseudo-random, avergae swaps needed: 1450.09092

Q3
Mean Words: 839
Algo score, swaps needed: 1040.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1536, 1489, 1343, 1626, 1411, 1448, 1468, 1566, 1453, 1659]
Pseudo-random, avergae swaps needed: 1458.6994

../data/crim_instructor_E

['ID', 'Q1', 'Q2']
leaving z scores in place
leaving z scores in place


,Q1_x,Q2_x,Q1_y,Q2_y
0,2.307804,-1.091625,1.138963,0.822366
1,-1.057500,-0.513174,0.247123,0.852983
2,0.400798,2.571898,-0.279174,-0.654121
3,-1.506207,-0.513174,0.616142,-0.482805
4,0.512975,0.450911,0.967134,0.741710


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1755.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2003, 1965, 1866, 2071, 1996, 2199, 2083, 2266, 1960, 2092]
Pseudo-random, avergae swaps needed: 2045.10629

Q2
Mean Words: 2137
Algo score, swaps needed: 1796.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1842, 1895, 1726, 1944, 1933, 2004, 1832, 1911, 1720, 1965]
Pseudo-random, avergae swaps needed: 1994.30083

Swaps (Machine):
[1261.0, 1261.0, 1165.0, 1571.0, 1500.0, 101.0, 891.0, 1100.99688, 1337.0, 1112.0, 1040.0, 1755.0, 1796.0]
N: 13 	Mean:  1222.3843753846154 	Var:  173710.14110554324
Avergae swaps (Pseudo-random):
[1499.08496, 1566.46133, 1560.01991, 1837.60356, 1757.76941, 182.53653, 1324.54328, 1219.78771, 1472.20256, 1450.09092, 1458.6994, 2045.10629, 1994.30083]
N: 13 	Mean:  1489.8620530769233 	Var:  198350.99757177386 

1 ) 1499.08496 > 1261.0 => 17.25200227169818
2 ) 1566.46133 > 1261.0 => 21.60675562625644
3 ) 1560.01

In [21]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,87,70,87
1,9,30,22,77,87,80
2,6,27,9,97,70,87
3,1,1,4,83,80,83
4,13,26,25,87,73,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1582.81408
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1455, 1317, 1254, 1616, 1568, 1321, 1306, 1534, 1571, 1578]
Pseudo-random, avergae swaps needed: 1499.49627

Q1
Mean Words: 2048
Algo score, swaps needed: 1485.08104
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1477, 1538, 1624, 1220, 1631, 1435, 1458, 1401, 1565, 1456]
Pseudo-random, avergae swaps needed: 1565.52429

Q2
Mean Words: 947
Algo score, swaps needed: 1520.66709
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1628, 1559, 1686, 1611, 1633, 1513, 1647, 1610, 1652, 1604]
Pseudo-random, avergae swaps needed: 1560.11127

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,83,87
1,18.0,20.0,63,67
2,16.5,18.0,70,83
3,18.0,22.5,77,90
4,24.0,22.5,90,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1566.52566
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1996, 1747, 1877, 1803, 1784, 2041, 1679, 1837, 1723, 1781]
Pseudo-random, avergae swaps needed: 1836.9894

Q2
Mean Words: 1546
Algo score, swaps needed: 1489.90083
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1971, 1820, 1850, 1948, 1634, 1904, 1979, 1579, 1451, 1854]
Pseudo-random, avergae swaps needed: 1757.72914

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 875.47784
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1430, 1498, 1216, 1503, 1246, 1474, 1415, 1368, 1295, 1403]
Pseudo-random, avergae swaps needed: 1324.44882

Q2
Mean Words: 570
Algo score, swaps needed: 946.0439
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1278, 1194, 1228, 1348, 1122, 1323, 1445, 1354, 1228, 1132]
Pseudo-random, avergae swaps needed: 1219.65549

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,87,97
1,44,29,30,77,73,80
2,38,36,20,87,83,83
3,47,16,21,60,67,59
4,38,25,31,83,83,90


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1298.45647
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1530, 1410, 1202, 1343, 1313, 1538, 1456, 1504, 1538, 1320]
Pseudo-random, avergae swaps needed: 1472.55968

Q2
Mean Words: 1050
Algo score, swaps needed: 1141.50078
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1503, 1561, 1429, 1561, 1320, 1428, 1467, 1624, 1476, 1689]
Pseudo-random, avergae swaps needed: 1450.79574

Q3
Mean Words: 839
Algo score, swaps needed: 1101.52601
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1370, 1403, 1550, 1421, 1493, 1407, 1370, 1397, 1431, 1433]
Pseudo-random, avergae swaps needed: 1458.17969

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,83,87
2,20.0,29.0,77,80
3,11.5,13.0,83,80
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1797.92638
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1928, 2044, 2180, 2074, 2009, 1787, 2289, 1831, 1973, 2126]
Pseudo-random, avergae swaps needed: 2045.56481

Q2
Mean Words: 2137
Algo score, swaps needed: 1768.99405
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2379, 1765, 1837, 2183, 2257, 1911, 2143, 2025, 1742, 1949]
Pseudo-random, avergae swaps needed: 1994.21542

Swaps (Machine):
[1582.81408, 1485.08104, 1520.66709, 1566.52566, 1489.90083, 96.48784, 875.47784, 946.0439, 1298.45647, 1141.50078, 1101.52601, 1797.92638, 1768.99405]
N: 13 	Mean:  1282.4155361538465 	Var:  196335.31936888624
Avergae swaps (Pseudo-random):
[1499.49627, 1565.52429, 1560.11127, 1836.9894, 1757.72914, 182.6587, 1324.44882, 1219.65549, 1472.55968, 1450.79574, 1458.17969, 2045.56481, 1994.21542]
N: 13 	Mean:  1489.8406707692307 	Var:  198320.27958058935 

1 ) 1499.49627 > 1582.81408 => -5.406192144149275

[back to contents](#Contents)